### Video Processing Pipeline YouTube

This pipeline downloads videos from YouTube, processes video frames to detect objects using a machine learning model, and saves the results. The pipeline is structured into modular classes for better readability, maintainability, and scalability.

#### Summary

1.  **Initialization**: Set up the Roboflow model.
2.  **Video Downloading**: Download videos from YouTube based on a search query.
3.  **Frame Processing**: Process each frame of the downloaded videos for object detection.
4.  **Logging and Progress Tracking**: Use logging for tracking progress and errors, and `tqdm` for progress bars.

#### Functionalities

1.  **RoboflowModel Class**: Initializes the Roboflow model.
2.  **VideoDownloader Class**: Downloads videos from YouTube based on a search query.
3.  **FrameProcessor Class**: Processes video frames for object detection and saves the results.

### Code Sections

1.  **Setup Logging and Imports**
2.  **Define the `RoboflowModel` Class**
3.  **Define the `VideoDownloader` Class**
4.  **Define the `FrameProcessor` Class**
5.  **Main Function and Execution**

----------

#### 1. Setup Logging and Imports

First, we need to import the necessary libraries and set up logging for tracking purposes.

In [2]:
import logging
import os
import cv2
from roboflow import Roboflow
import supervision as sv
import youtube_dl
from tqdm import tqdm
import yt_dlp as youtube_dl

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


#### 2. Define the `RoboflowModel` Class

This class initializes the Roboflow model using an API key.

In [4]:
class RoboflowModel:
    def __init__(self, api_key):
        self.api_key = api_key
        self.model = None

    def initialize_model(self):
        logging.info("Initializing Roboflow...")
        rf = Roboflow(api_key=self.api_key)

        logging.info("Loading workspace...")
        workspace = rf.workspace()

        logging.info("Loading project...")
        project = workspace.project("license-plate-nmu02")

        logging.info("Loading model...")
        self.model = project.version(1).model

        logging.info("Model loaded successfully")
        return self.model


#### 3. Define the `VideoDownloader` Class

This class handles the downloading of videos from YouTube based on a search query.

In [6]:
class VideoDownloader:
    def __init__(self, search_query, num_videos=5):
        self.search_query = search_query
        self.num_videos = num_videos
        self.video_dir = 'downloaded_videos'
        self.video_paths = []

    def create_directory(self, path):
        os.makedirs(path, exist_ok=True)

    def search_videos(self):
        logging.info("Searching for videos...")
        ydl_opts = {
            'quiet': True,
            'extract_flat': True,
            'skip_download': True
        }
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(f"ytsearch{self.num_videos}:{self.search_query}", download=False)
            return result['entries']

    def download_videos(self):
        self.create_directory(self.video_dir)
        logging.info(f"Downloading {self.num_videos} videos to {self.video_dir}...")

        videos = self.search_videos()

        for i, video in enumerate(tqdm(videos, desc="Downloading videos")):
            url = video['url']
            ydl_opts = {
                'format': 'mp4',
                'outtmpl': os.path.join(self.video_dir, f"video_{i + 1}.mp4"),
                'quiet': True
            }
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(url, download=True)
                file_path = ydl.prepare_filename(info_dict)
            self.video_paths.append(file_path)
            logging.info(f"Downloaded video {i + 1}: {video['title']}")

        return self.video_paths



#### 4. Define the `FrameProcessor` Class

This class processes the frames of downloaded videos for object detection and saves the results.

In [8]:
class FrameProcessor:
    def __init__(self, model):
        self.model = model

    def create_directory(self, path):
        os.makedirs(path, exist_ok=True)

    def process_video_frames(self, video_path):
        video_name = os.path.basename(video_path).split('.')[0]
        output_image_directory = f'output/{video_name}/images'
        output_label_directory = f'output/{video_name}/labels'

        self.create_directory(output_image_directory)
        self.create_directory(output_label_directory)

        video_capture = cv2.VideoCapture(video_path)
        frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

        logging.info(f"Processing frames of video {video_name}...")

        for frame_num in tqdm(range(frame_count), desc=f"Processing {video_name}"):
            ret, frame = video_capture.read()
            if not ret:
                break

            temp_image_path = os.path.join(output_image_directory, f"temp_frame_{frame_num + 1}.jpg")
            cv2.imwrite(temp_image_path, frame)

            try:
                result = self.model.predict(temp_image_path, confidence=40, overlap=30).json()
            except Exception as e:
                logging.error(f"Error in prediction for frame {frame_num + 1} of video {video_name}: {e}")
                continue

            if result["predictions"]:
                final_image_path = os.path.join(output_image_directory, f"frame_{frame_num + 1}.jpg")
                os.rename(temp_image_path, final_image_path)

                label_file_path = os.path.join(output_label_directory, f"frame_{frame_num + 1}.txt")
                self.write_labels(label_file_path, result["predictions"], frame.shape)
            else:
                os.remove(temp_image_path)
                logging.info(f"No detections in frame {frame_num + 1} of video {video_name}. Image not saved.")

        video_capture.release()
        cv2.destroyAllWindows()

    def write_labels(self, label_file_path, predictions, frame_shape):
        with open(label_file_path, 'w') as label_file:
            for prediction in predictions:
                class_id = prediction["class"]
                x_center = prediction["x"] / frame_shape[1]
                y_center = prediction["y"] / frame_shape[0]
                width = prediction["width"] / frame_shape[1]
                height = prediction["height"] / frame_shape[0]
                label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


#### 5. Main Function and Execution

The main function initializes the classes and executes the pipeline.

In [ ]:
def main():
    api_key = "TXHvXfRbZfimcC9mjhFw"
    roboflow_model = RoboflowModel(api_key)
    model = roboflow_model.initialize_model()

    search_query = input("Enter a search string for YouTube videos: ")
    video_downloader = VideoDownloader(search_query)
    video_paths = video_downloader.download_videos()

    frame_processor = FrameProcessor(model)
    for video_path in video_paths:
        frame_processor.process_video_frames(video_path)

    logging.info("Processing completed.")

if __name__ == "__main__":
    main()

2024-07-09 03:03:48,946 - INFO - Initializing Roboflow...
2024-07-09 03:03:49,452 - INFO - Loading workspace...


loading Roboflow workspace...


2024-07-09 03:03:49,918 - INFO - Loading project...


loading Roboflow project...


2024-07-09 03:03:50,298 - INFO - Loading model...
2024-07-09 03:03:50,708 - INFO - Model loaded successfully


...

2024-07-09 03:04:17,051 - INFO - Downloaded video 1: Traffic - ട്രാഫിക് Malayalam Full Movie || Sreenivasan, Kunchacko Boban || TVNXT Malayalam

2024-07-09 03:04:22,488 - INFO - Downloaded video 2: Incredible traffic jam in Dhaka ,Bangladesh। Dhanmondi-27.

2024-07-09 03:04:26,007 - INFO - Downloaded video 3: Kannerinjal | Traffic | Vipin Xavier | Hisham | Mejjo Josseph | S Ramesan Nair

2024-07-09 03:04:32,518 - INFO - Downloaded video 4: bombay traffic

2024-07-09 03:04:35,679 - INFO - Downloaded video 5: The Simple Solution to Traffic
2024-07-09 03:04:35,713 - INFO - Processing frames of video video_1...
Processing video_1:   0%|          | 78/168231 [01:00<45:46:44,  1.02it/s]